### Import and configure packages

In [81]:
import panel as pn
import numpy as np
import pandas as pd
pn.extension('tabulator', sizing_mode="stretch_width")
import hvplot.pandas

### Load data

In [3]:
# Load & clean data, and make DataFrame interactive()

colores = ['#29aec1', '#7b5b32', '#559900', '#301e02']

df = pd.read_csv('Banff Diverted Materials 2017-2021.csv')
df = df[df.Materials != 'Clean Cover'].reset_index(drop=True)
idf = df.interactive()

### Define panel widgets

In [69]:
flow_choice_box = pn.widgets.CheckBoxGroup(
    name='Materials Flow', options=['Incoming', 'Outgoing'], 
    value=['Incoming', 'Outgoing'])

#flow_choice_box

CheckBoxGroup(name='Materials Flow', options=['Incoming', 'Outgoing'], sizing_mode='stretch_width', value=['Incoming', 'Outgoing'])

### Define pipeplines and plots

In [70]:
# Total material flow data pipeline

tot_flow = idf[(idf['Flow'].isin(flow_choice_box))].groupby(['Year','Flow', 'End'], as_index=False).sum()

In [72]:
# Material Flow.

tot_flow1 = idf[(idf['Flow'].isin(flow_choice_box))].groupby(['Year','Flow'], as_index=False).sum()

tf1 = tot_flow1.hvplot(x='Year', y='Tonnes', by='Flow', min_height=400, xticks=5, padding=0.1,
                 color=['#ff8f00', '#6a329f'], line_width=4, title='Material flow at Banff transfer station', grid=True, legend='top_right',
                 max_width=400, kind='line', responsive=True, ylim=(0, 9000))
tf11 = tot_flow1.hvplot(x='Year', y='Tonnes', by='Flow', min_height=400, responsive=True, xticks=5,
                 color=['#ff8f00', '#6a329f'], size=300, padding=0.1, xlabel='', kind='scatter',ylim=(0, 9000))
total_flow_plot = tf1 * tf11
#total_flow_plot

In [73]:
flow_bar = tot_flow.hvplot(x='Year', y='Tonnes', by='End', padding=0.1,
                           color=colores, rot=45, kind='bar', responsive=True,
                           title='Material flow at Banff transfer station', 
                           grid=True, xlabel='', min_height=400,)
#flow_bar

In [74]:
# Materials flowing to end of life process

fl = tot_flow.hvplot(x='Year', y='Tonnes', by='End', kind='line', min_height=400, xticks=5, padding=0.1,
                 color=colores, line_width=4, title='Material flow at Banff transfer station', 
                 grid=True, legend='top_right', responsive=True, ylim=(0, 9000))
fl1 = tot_flow.hvplot(x='Year', y='Tonnes', by='End', kind='scatter', min_height=400, xticks=5,
                 color=colores, size=300, padding=0.1, xlabel='', legend=False, responsive=True,
                 ylim=(0, 9000))
flow_line = fl * fl1
#flow_line

In [76]:
# Filter dataset by Incoming material, Outgoing material, Food

df_incoming = df[df.Flow == 'Incoming'].reset_index(drop=True)
df_outgoing = df[df.Flow == 'Outgoing'].reset_index(drop=True)
food = df[df['Materials'] == 'Food']
food = food[['Year', 'Tonnes']].groupby('Year')['Tonnes'].agg(Food='sum').round().reset_index()
food = food.set_index('Year')

35290.0
30993.0
3309.0


In [77]:
# Totals material by year

mat_total_in = df_incoming.groupby('Year')['Tonnes'].agg(Total_incoming='sum').round().reset_index()

# Mataterial diverted

fil = (df_incoming['End'] == 'To be Diverted')
mat_diverted_in = df_incoming[fil].groupby('Year')['Tonnes'].agg(To_be_Diverted='sum').round().reset_index()

# Material Landfilled

fil2 = (df_incoming['End'] == 'To be Landfilled')
mat_landfilled_in = df_incoming[fil2].groupby('Year')['Tonnes'].agg(To_be_Landfilled='sum').round().reset_index()

# Materiales por año

mat_year_in = pd.merge(mat_diverted_in , mat_landfilled_in,  on='Year')
mat_year_in = pd.merge(mat_year_in, mat_total_in, on='Year')
mat_year_in = mat_year_in.set_index('Year')
mat_year_in = pd.concat([mat_year_in, food], axis=1)
mat_year_in['Diversion_rate'] = ((mat_year_in.To_be_Diverted / mat_year_in.Total_incoming) * 100).round()
mat_year_in['Incoming waste'] = ((mat_year_in.To_be_Landfilled / mat_year_in.Total_incoming) * 100).round()
mat_year_in['Food diversion rate'] = ((mat_year_in.Food / mat_year_in.Total_incoming) * 100).round()
mat_year_in['Diversion rate incoming'] = (mat_year_in['Diversion_rate'] - mat_year_in['Food diversion rate'])
mat_year_in['Div'] = 'Incoming Diversion'
#mat_year_in

In [41]:
# Totals material by year

mat_total_out = df_outgoing.groupby('Year')['Tonnes'].agg(Total_Outgoing='sum').round().reset_index()

# Mataterial diverted

fil = (df_outgoing['End'] == 'Diverted')
mat_diverted_out = df_outgoing[fil].groupby('Year')['Tonnes'].agg(Diverted='sum').round().reset_index()

# Material Landfilled

fil2 = (df_outgoing['End'] == 'Landfilled')
mat_landfilled_out = df_outgoing[fil2].groupby('Year')['Tonnes'].agg(Landfilled='sum').round().reset_index()

# Materiales por año

mat_year_out = pd.merge(mat_diverted_out , mat_landfilled_out,  on='Year')
mat_year_out = pd.merge(mat_year_out, mat_total_out, on='Year')
mat_year_out = mat_year_out.set_index('Year')
mat_year_out['Diversion rate outgoing'] = ((mat_year_out.Diverted / mat_year_out.Total_Outgoing) * 100).round()
mat_year_out['Outgoing Landfilled waste'] = ((mat_year_out.Landfilled / mat_year_out.Total_Outgoing) * 100).round()
mat_year_out['Div'] = 'Outgoing Diversion'
#mat_year_out

In [42]:
mat_div = pd.concat([mat_year_in , mat_year_out])
#mat_div

In [43]:
#mat_div.rename(columns={'To_be_Diverted': 'To be Diverted', 'To_be_Landfilled': 'To be Landfilled', 
#                        'Diversion_rate': "Diversion rate total", 'Total_incoming': }, inplace=True)
#.columns

,To_be_Diverted,To_be_Landfilled,Total_incoming,Food,Diversion_rate,Incoming waste,Food diversion rate,Diversion rate incoming,Div,Diverted,Landfilled,Total_Outgoing,Diversion rate outgoing,Outgoing Landfilled waste
Year,,,,,,,,,,,,,,
2017,2194.0,5187.0,7381.0,450.0,30.0,70.0,6.0,24.0,Incoming Diversion,NaN,NaN,NaN,NaN,NaN
2018,2714.0,5067.0,7781.0,720.0,35.0,65.0,9.0,26.0,Incoming Diversion,NaN,NaN,NaN,NaN,NaN
2019,3049.0,5088.0,8137.0,949.0,37.0,63.0,12.0,25.0,Incoming Diversion,NaN,NaN,NaN,NaN,NaN
2020,2325.0,3666.0,5991.0,543.0,39.0,61.0,9.0,30.0,Incoming Diversion,NaN,NaN,NaN,NaN,NaN
2021,2429.0,3570.0,6000.0,647.0,40.0,60.0,11.0,29.0,Incoming Diversion,NaN,NaN,NaN,NaN,NaN
2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outgoing Diversion,2764.0,3711.0,6475.0,43.0,57.0
2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outgoing Diversion,2436.0,2780.0,5216.0,47.0,53.0
2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outgoing Diversion,4525.0,3418.0,7943.0,57.0,43.0
2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outgoing Diversion,3730.0,2350.0,6080.0,61.0,39.0


In [45]:
imat_div = mat_div.interactive()

In [46]:
div_togg = pn.widgets.RadioButtonGroup(
    name='Diversion', options=['Incoming Diversion', 'Outgoing Diversion'], 
    button_type='primary',sizing_mode="stretch_width", orientation='vertical')
#div_togg

RadioButtonGroup(button_type='primary', name='Diversion', options=['Incoming Diversion', ...], orientation='vertical', sizing_mode='stretch_width', value='Incoming Diversion')

In [47]:
imat_div = imat_div[(imat_div['Div'] == div_togg)]

In [48]:
#imat_div

In [66]:
wgt_in = imat_div.hvplot(x= 'Year', y=['To_be_Diverted', 'Total_incoming','Diverted', 'Total_Outgoing'], kind='bar',
                         value_label='Tonnes', legend='bottom', rot=45, grid=True, responsive=True,
                         title='Banff solid waste diversion and disposal', min_height=300,
                         xlabel='', color=['#29aec1', '#ff8f00','#29aec1',  '#6a329f'])
#wgt_in

In [63]:
div_in = imat_div.hvplot(x= 'Year', y=['Diversion_rate', 'Incoming waste', 'Diversion rate outgoing', 'Outgoing Landfilled waste'], 
                         kind='bar', value_label='Percentage', 
                         grid=True, legend='bottom', stacked=True,
                         title='Banff Diversion Rate', xlabel='', 
                         color=['#29aec1', '#ff8f00', '#29aec1',  '#6a329f'])
#div_in

### Creating Dashboard

In [79]:
template2 = pn.template.FastGridTemplate(
    site='',
    logo='logo_town_banff.png',
    title='Banff solid waste diversion and disposal',
#    prevent_collision=True,
#    accent_base_color="#88d8b0",
    header_background='#2F4F4F',
#    background_color="#e5eeda",
    theme_toggle=True,
    corner_radius=15,
    sidebar_width=200,
    row_height=100,
    theme="dark"
)
template2.sidebar[:]=[pn.pane.PNG('take-the-zero-waste-trail.png', height=160, align='center'),
                      pn.pane.Markdown('####Banff is a model environmental community where everything we do is guided by a goal to preserve this special place. And reducing and diverting waste to landfill is critical to maintaining the well-being and sustainability of our community.',
                                       sizing_mode='stretch_width'),]
template2.main[:1, :2] = flow_choice_box
template2.main[:1, 2:10] = '## Looking at the flow of materials at Banff Transfer Station'
template2.main[1:5, :5] = flow_line.holoviews()
template2.main[1:5, 5:10] = total_flow_plot.holoviews()
template2.main[5:6, :8] = '## Explore Incoming and Outgoing diverion rates'
template2.main[5:6, 8:10] = div_togg
template2.main[6:9, :5] = div_in.holoviews()
template2.main[6:9, 5:10] = wgt_in.holoviews()
template2.servable()
#template2.show()

#template2.main[4:8, :6] = total_flow_plot.holoviews()
#template2.main[4:8, 6:] = flow_bar.holoviews()

Launching server at http://localhost:52924
